In [ ]:
import os, json, time, joblib, numpy as np, pandas as pd
from pathlib import Path
np.random.seed(42)


FAST = True            
N_FOLDS_FAST = 3       # folds para ranking
MAX_ITER_LR = 1000     # iters para Logistic
N_JOBS = -1


DATA = Path("../data")
MODELS_BASE = Path("../models/baseline");  MODELS_BASE.mkdir(parents=True, exist_ok=True)
MODELS_OUT  = Path("../models/experiments"); MODELS_OUT.mkdir(parents=True, exist_ok=True)
REPORTS     = Path("../reports"); REPORTS.mkdir(exist_ok=True)


df = pd.read_csv(DATA/"processed.csv")                # requiere 'text_norm'
with open(DATA/"classes.txt") as f: classes = [l.strip() for l in f if l.strip()]
with open(DATA/"splits.json") as f: splits = json.load(f)


tfidf_path = MODELS_BASE/"tfidf.joblib"
if not tfidf_path.exists():
    raise FileNotFoundError("No encuentro ../models/baseline/tfidf.joblib (corre 02_preprocess primero).")
tfidf = joblib.load(tfidf_path)


X = tfidf.transform(df["text_norm"].astype(str))
Y = df[classes].values.astype(int)


test_idx = np.array(splits["test_idx"])
trainval_idx = np.setdiff1d(np.arange(len(df)), test_idx)

len(df), X.shape, classes


(3563,
 (3563, 23621),
 ['cardiovascular', 'hepatorenal', 'neurological', 'oncological'])

In [ ]:

from sklearn.metrics import f1_score, roc_auc_score, average_precision_score
from sklearn.preprocessing import MinMaxScaler

def sweep_thresholds(y_true, y_score, grid=np.arange(0.05, 0.96, 0.01)):
    """Busca umbral por clase maximizando F1 (rápido, robusto)."""
    C = y_true.shape[1]; thr = np.zeros(C)
    for j in range(C):
        s = y_score[:, j]; yt = y_true[:, j]
        best_f1, best_t = -1, 0.5
        for t in grid:
            f1 = f1_score(yt, (s >= t).astype(int), zero_division=0)
            if f1 > best_f1: best_f1, best_t = f1, t
        thr[j] = best_t
    return thr

def to_proba(model, Xsub):
    """
    Devuelve probabilidades para multilabel.
    - Si hay predict_proba: usa proba[:,1] por clase (OVR).
    - Si hay decision_function: reescala a [0,1] con MinMax (rápido, evita calibración pesada).
    """
    if hasattr(model, "predict_proba"):
        p = model.predict_proba(Xsub)
        if isinstance(p, list):  # OVR con lista por clase
            p = np.vstack([col[:, 1] for col in p]).T
        return p
    scores = model.decision_function(Xsub)
    return MinMaxScaler().fit_transform(scores)

def metrics_from(y_true, y_score, thr):
    y_pred = (y_score >= thr).astype(int)
    m = {
        "f1_micro": f1_score(y_true, y_pred, average="micro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "roc_auc_macro": roc_auc_score(y_true, y_score, average="macro"),
        "pr_auc_macro":  average_precision_score(y_true, y_score, average="macro"),
    }
    # detalle por clase (útil para informe)
    m["per_class"] = {}
    for i, c in enumerate(classes):
        m["per_class"][c] = {
            "f1": f1_score(y_true[:,i], y_pred[:,i], zero_division=0),
            "roc_auc": roc_auc_score(y_true[:,i], y_score[:,i]),
            "pr_auc": average_precision_score(y_true[:,i], y_score[:,i]),
            "thr": float(thr[i]),
        }
    return m, y_pred

def save_json(obj, path):
    with open(path, "w") as f: json.dump(obj, f, indent=2)


In [ ]:

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import ComplementNB
from sklearn.multiclass import OneVsRestClassifier

def get_models():
    return {
        # baseline fuerte
        "logreg_C2": OneVsRestClassifier(LogisticRegression(
            solver="saga", penalty="l2", C=2.0, max_iter=MAX_ITER_LR,
            n_jobs=N_JOBS, class_weight="balanced", random_state=42
        ), n_jobs=N_JOBS),

        # muy competitivo con TF-IDF
        "ridge_a1": OneVsRestClassifier(RidgeClassifier(
            alpha=1.0, class_weight="balanced", random_state=42
        ), n_jobs=N_JOBS),

        # SVM lineal sin calibración 
        "linsvm_C1": OneVsRestClassifier(LinearSVC(
            C=1.0, random_state=42
        ), n_jobs=N_JOBS),

        # Naive Bayes complement 
        "cnb_a05": OneVsRestClassifier(ComplementNB(alpha=0.5), n_jobs=N_JOBS),
    }


In [ ]:
from sklearn.base import clone
import numpy as np

FOLDS_FAST  = [ (np.array(f["train_idx"]), np.array(f["val_idx"])) for f in splits["folds"][:N_FOLDS_FAST] ]
FOLDS_FULL  = [ (np.array(f["train_idx"]), np.array(f["val_idx"])) for f in splits["folds"] ]

def cv_experiment(name, model, folds):
    t0 = time.time()
    fold_thrs, fold_metrics = [], []

    for tr, va in folds:
        mdl = clone(model)
        mdl.fit(X[tr], Y[tr])
        proba = to_proba(mdl, X[va])
        thr = sweep_thresholds(Y[va], proba)
        m,_ = metrics_from(Y[va], proba, thr)
        fold_thrs.append(thr); fold_metrics.append(m)

    thr_med = np.median(np.array(fold_thrs), axis=0)


    mdl_final = clone(model).fit(X[trainval_idx], Y[trainval_idx])
    test_proba = to_proba(mdl_final, X[test_idx])
    test_m, _ = metrics_from(Y[test_idx], test_proba, thr_med)

    elapsed = round((time.time() - t0)/60, 2)
    res = {
        "name": name,
        "cv_f1_micro": float(np.mean([m["f1_micro"] for m in fold_metrics])),
        "cv_f1_macro": float(np.mean([m["f1_macro"] for m in fold_metrics])),
        "test_f1_micro": test_m["f1_micro"],
        "test_f1_macro": test_m["f1_macro"],
        "minutes": elapsed,
        "thr": thr_med.tolist(),
    }
    return res, mdl_final, thr_med, test_proba, test_m


In [ ]:
RUN = ["logreg_C2", "ridge_a1", "linsvm_C1", "cnb_a05"]  

folds = FOLDS_FAST if FAST else FOLDS_FULL
results, artifacts = [], {}

for name in RUN:
    print(f"\n>>> {name}  ({'FAST' if FAST else 'FULL'})")
    res, mdl, thr, test_proba, test_m = cv_experiment(name, get_models()[name], folds)
    results.append(res)
    artifacts[name] = {"model": mdl, "thr": np.array(thr), "test_proba": test_proba, "test_metrics": test_m}
    pd.DataFrame(results).to_csv(REPORTS/"experiments_rank_partial.csv", index=False)

rank_df = pd.DataFrame(results).sort_values("test_f1_macro", ascending=False)
display(rank_df)
rank_df.to_csv(REPORTS/"experiments_rank.csv", index=False)



>>> logreg_C2  (FAST)

>>> ridge_a1  (FAST)

>>> linsvm_C1  (FAST)

>>> cnb_a05  (FAST)


,name,cv_f1_micro,cv_f1_macro,test_f1_micro,test_f1_macro,minutes,thr
2,linsvm_C1,0.897535,0.890769,0.886076,0.870591,0.06,"[0.36000000000000004, 0.34, 0.4700000000000001..."
0,logreg_C2,0.887863,0.880181,0.868505,0.855164,0.18,"[0.4600000000000001, 0.4600000000000001, 0.510..."
1,ridge_a1,0.891695,0.883198,0.876933,0.848997,0.05,"[0.4100000000000001, 0.38000000000000006, 0.51..."
3,cnb_a05,0.818879,0.776630,0.807575,0.758549,0.04,"[0.5000000000000001, 0.43000000000000005, 0.05..."


In [ ]:
TOP_K = min(2, len(rank_df))
candidates = rank_df.head(TOP_K)["name"].tolist()
print("Top-2 para tuning:", candidates)

param_grid = {
    "logreg_C2": [{"C": c} for c in [0.5, 1.0, 2.0, 4.0]],
    "ridge_a1" : [{"alpha": a} for a in [0.5, 1.0, 2.0]],
    "linsvm_C1":[{"C": c} for c in [0.5, 1.0, 2.0]],
    "cnb_a05"  : [{"alpha": a} for a in [0.1, 0.5, 1.0]],
}

def set_params(base, p):
    """Crea una copia del modelo de get_models con hiperparámetros p."""
    from sklearn.base import clone
    m = get_models()[base]
    est = clone(m.estimator)
    est.set_params(**p)
    return m.set_params(estimator=est)

tune_res = []
for base in candidates:
    grid = param_grid.get(base, [{}])
    for p in grid:
        name = f"{base}_{'_'.join([f'{k}{v}' for k,v in p.items()])}"
        res, mdl, thr, test_proba, test_m = cv_experiment(name, set_params(base, p), FOLDS_FAST)
        tune_res.append(res)

tune_df = pd.DataFrame(tune_res).sort_values("test_f1_macro", ascending=False)
display(tune_df)
tune_df.to_csv(REPORTS/"experiments_tuning.csv", index=False)


Top-2 para tuning: ['linsvm_C1', 'logreg_C2']


,name,cv_f1_micro,cv_f1_macro,test_f1_micro,test_f1_macro,minutes,thr
0,linsvm_C1_C0.5,0.896595,0.890018,0.895086,0.886949,0.05,"[0.36000000000000004, 0.35000000000000003, 0.4..."
1,linsvm_C1_C1.0,0.897535,0.890769,0.886076,0.870591,0.05,"[0.36000000000000004, 0.34, 0.4700000000000001..."
2,linsvm_C1_C2.0,0.896414,0.889009,0.882548,0.865607,0.05,"[0.38000000000000006, 0.34, 0.4600000000000001..."
6,logreg_C2_C4.0,0.890892,0.882074,0.875800,0.865032,0.12,"[0.44000000000000006, 0.42000000000000004, 0.5..."
4,logreg_C2_C1.0,0.878568,0.872639,0.868096,0.856712,0.06,"[0.4600000000000001, 0.4700000000000001, 0.520..."
5,logreg_C2_C2.0,0.887863,0.880181,0.868505,0.855164,0.08,"[0.4600000000000001, 0.4600000000000001, 0.510..."
3,logreg_C2_C0.5,0.863552,0.858972,0.853033,0.840248,0.05,"[0.4600000000000001, 0.4700000000000001, 0.460..."


In [ ]:
TOP_ENSEMBLE = 3
top_names = rank_df.head(TOP_ENSEMBLE)["name"].tolist()
print("Para ensemble:", top_names)
ens = []
for name in top_names:
    if name in artifacts:
        ens.append(artifacts[name])
    else:
        res, mdl, thr, test_proba, test_m = cv_experiment(name, get_models()[name], FOLDS_FAST)
        ens.append({"model": mdl, "thr": np.array(thr), "test_proba": test_proba, "test_metrics": test_m})

probas = [e["test_proba"] for e in ens]
P = np.mean(probas, axis=0)
THR_ENS = np.median(np.vstack([e["thr"] for e in ens]), axis=0)

m_ens, y_pred_ens = metrics_from(Y[test_idx], P, THR_ENS)
print("ENSEMBLE — test_f1_micro:", round(m_ens["f1_micro"],4), " test_f1_macro:", round(m_ens["f1_macro"],4))

save_json(m_ens, REPORTS/"ensemble_test_metrics.json")
save_json({c: float(t) for c,t in zip(classes, THR_ENS)}, MODELS_OUT/"ensemble_thresholds.json")


Para ensemble: ['linsvm_C1', 'logreg_C2', 'ridge_a1']
ENSEMBLE — test_f1_micro: 0.8882  test_f1_macro: 0.8795


In [ ]:
import re
from sklearn.base import clone

def resolve_base_key(name: str):
    keys = sorted(get_models().keys(), key=len, reverse=True)
    for k in keys:
        if name.startswith(k):
            return k
    raise KeyError(f"No encuentro clave base para '{name}'")

def parse_params_from_name(name: str, base_key: str):
    suffix = name[len(base_key):]
    if suffix.startswith('_'):
        suffix = suffix[1:]
    params = {}
    if suffix:
        for tok in suffix.split('_'):
            m = re.match(r'([A-Za-zA-Z]+)([-+]?\d*\.?\d+)$', tok)
            if m:
                k, v = m.groups()
                v = float(v)
                if v.is_integer():
                    v = int(v)
                params[k] = v
    return params

def set_params(base_key: str, params: dict):
    base = get_models()[base_key]
    est = clone(base.estimator)
    est.set_params(**params)
    return base.set_params(estimator=est)

best_single = tune_df.iloc[0] if 'tune_df' in globals() and len(tune_df) else rank_df.iloc[0]
print("Mejor single (tuning o rank):", best_single["name"])

base_key  = resolve_base_key(best_single["name"])
params    = {}

if 'tune_df' in globals() and 'params' in tune_df.columns:
    row = tune_df[tune_df["name"] == best_single["name"]]
    if len(row):
        params = row.iloc[0].get("params", {}) or {}
if not params:
    params = parse_params_from_name(best_single["name"], base_key)

model_for_full = set_params(base_key, params) if params else get_models()[base_key]

res_full, mdl_full, thr_full, test_proba_full, test_m_full = cv_experiment(
    best_single["name"], model_for_full, FOLDS_FULL
)
print("FULL — test_f1_micro:", round(res_full["test_f1_micro"],4),
      " test_f1_macro:", round(res_full["test_f1_macro"],4))

winner_name, winner_obj, winner_thr, winner_metrics = None, None, None, None
if 'm_ens' in globals() and m_ens:
    if m_ens["f1_macro"] >= res_full["test_f1_macro"]:
        winner_name = "ensemble_blend"
        winner_obj  = None
        winner_thr  = THR_ENS
        winner_metrics = m_ens
    else:
        winner_name = best_single["name"] + "_full5fold"
        winner_obj  = mdl_full
        winner_thr  = thr_full
        winner_metrics = test_m_full
else:
    winner_name = best_single["name"] + "_full5fold"
    winner_obj  = mdl_full
    winner_thr  = thr_full
    winner_metrics = test_m_full

print("GANADOR:", winner_name, "| macro-F1:", round(winner_metrics["f1_macro"],4))

win_dir = MODELS_OUT / winner_name
win_dir.mkdir(parents=True, exist_ok=True)

if winner_obj is not None:
    joblib.dump(winner_obj, win_dir/"model.joblib")
save_json({c: float(t) for c,t in zip(classes, winner_thr)}, win_dir/"thresholds.json")
save_json(winner_metrics, REPORTS/f"{winner_name}_test_metrics.json")

print("Artefactos en:", win_dir)
winner_name, winner_metrics


Mejor single (tuning o rank): linsvm_C1_C0.5
FULL — test_f1_micro: 0.8933  test_f1_macro: 0.8855
GANADOR: linsvm_C1_C0.5_full5fold | macro-F1: 0.8855
Artefactos en: ..\models\experiments\linsvm_C1_C0.5_full5fold


('linsvm_C1_C0.5_full5fold',
 {'f1_micro': 0.8932547478716437,
  'f1_macro': 0.8854984194294601,
  'roc_auc_macro': np.float64(0.9618312768809928),
  'pr_auc_macro': np.float64(0.9432351466086362),
  'per_class': {'cardiovascular': {'f1': 0.9158415841584159,
    'roc_auc': np.float64(0.9768150196761656),
    'pr_auc': np.float64(0.9706234965926278),
    'thr': 0.35000000000000003},
   'hepatorenal': {'f1': 0.8672566371681416,
    'roc_auc': np.float64(0.9521662547978339),
    'pr_auc': np.float64(0.9372755327073047),
    'thr': 0.35000000000000003},
   'neurological': {'f1': 0.9048414023372288,
    'roc_auc': np.float64(0.9716638330499716),
    'pr_auc': np.float64(0.9705105556858418),
    'thr': 0.4600000000000001},
   'oncological': {'f1': 0.8540540540540541,
    'roc_auc': np.float64(0.9466800000000001),
    'pr_auc': np.float64(0.8945310014487706),
    'thr': 0.36000000000000004}}})

In [ ]:

def predict_texts(texts, model_dir, is_ensemble=False, members=None):
    """
    is_ensemble=True -> usa average de probas de miembros 'members' (lista de directorios).
    """
    thr_map = json.load(open(Path(model_dir)/"thresholds.json"))
    thr = np.array([thr_map[c] for c in classes])

    X_new = tfidf.transform(pd.Series(texts).astype(str))

    if is_ensemble:
        Ps = []
        for mdir in members:
            mdl = joblib.load(Path(mdir)/"model.joblib")
            Ps.append(to_proba(mdl, X_new))
        proba = np.mean(Ps, axis=0)
    else:
        mdl = joblib.load(Path(model_dir)/"model.joblib")
        proba = to_proba(mdl, X_new)

    y_pred = (proba >= thr).astype(int)
    out = []
    for i, t in enumerate(texts):
        labs = [c for j,c in enumerate(classes) if y_pred[i,j]==1]
        out.append({
            "text": (t[:120] + "...") if len(t)>120 else t,
            "labels": labs,
            "proba": {c: float(proba[i,j]) for j,c in enumerate(classes)}
        })
    return out


if (MODELS_OUT/winner_name/"model.joblib").exists():
    predict_texts(
        ["randomized trial on cardiac arrhythmia and stroke risk in diabetic patients"],
        MODELS_OUT/winner_name
    )


In [ ]:

def top_terms_linear(model, k=20):
    """Extrae top-k términos por clase a partir de coeficientes (LR, Ridge, LinearSVC)."""
    est = model.estimators_[0] if hasattr(model, "estimators_") else None
    if est is None or not hasattr(est, "coef_"):
        return None
    vocab = np.array([t for t,_ in sorted(tfidf.vocabulary_.items(), key=lambda x: x[1])])
    out = {}
    for i, c in enumerate(classes):
        coef = model.estimators_[i].coef_.ravel()
        idx = np.argsort(coef)[-k:][::-1]
        out[c] = list(zip(vocab[idx], coef[idx].round(4)))
    return out

if (MODELS_OUT/winner_name/"model.joblib").exists():
    mdl = joblib.load(MODELS_OUT/winner_name/"model.joblib")
    terms = top_terms_linear(mdl, k=20)
    if terms is not None:
        for c, lst in terms.items():
            print(f"\nTop términos + para clase '{c}':")
            for w,v in lst: print(f"{w:25s} {v:7.4f}")



Top términos + para clase 'cardiovascular':
heart                      2.6502
cardiac                    2.5186
vascular insights          2.4374
vascular                   2.4286
markers in                 2.1236
myocardial                 1.9043
cardiovascular             1.8760
cardiomyopathy             1.8027
markers                    1.6658
aortic                     1.5940
valve                      1.5383
blood pressure             1.5151
cardiac connections        1.4645
pressure                   1.4511
hypotension                1.4165
hypotensive                1.3718
insights                   1.3532
myocardium                 1.3249
hypertensive               1.3134
hypertension               1.3075

Top términos + para clase 'hepatorenal':
renal                      4.3138
liver                      2.7520
hepatic                    2.3070
kidney                     2.2637
hepatocellular             1.8282
creatinine                 1.6470
nephritis                  1.

In [ ]:

import json, os, time, numpy as np, pandas as pd
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt

REPORTS = Path("../reports"); REPORTS.mkdir(exist_ok=True)
FIGS    = REPORTS/"figures"; FIGS.mkdir(parents=True, exist_ok=True)
MODELS_EXP = Path("../models/experiments")

if 'winner_name' not in globals():
    rank_csv = REPORTS/"experiments_rank.csv"
    assert rank_csv.exists(), "No encuentro experiments_rank.csv (corre el ranking primero)."
    rank_df = pd.read_csv(rank_csv).sort_values("test_f1_macro", ascending=False)
    winner_name = rank_df.iloc[0]["name"]
    print("Inferido winner_name:", winner_name)


if 'winner_metrics' not in globals():
    met_json = REPORTS/f"{winner_name}_test_metrics.json"
    if Path(met_json).exists():
        winner_metrics = json.load(open(met_json))
    else:
        raise FileNotFoundError(f"No encuentro {met_json}. Ejecuta la celda del finalista FULL.")


win_dir = MODELS_EXP/winner_name
thr_json = win_dir/"thresholds.json"
assert thr_json.exists(), f"No encuentro {thr_json}. Asegúrate de haber exportado los artefactos del ganador."
thr_map = json.load(open(thr_json))
thr_vec = np.array([thr_map[c] for c in classes])


y_test = Y[test_idx] if 'Y' in globals() else None
print("OK: ganador:", winner_name, "| thresholds:", list(thr_map.items())[:2], "...")


OK: ganador: linsvm_C1_C0.5_full5fold | thresholds: [('cardiovascular', 0.35000000000000003), ('hepatorenal', 0.35000000000000003)] ...


In [ ]:

per_class = pd.DataFrame.from_dict(winner_metrics["per_class"], orient="index").reset_index()
per_class = per_class.rename(columns={"index": "class"})
per_class = per_class[["class","f1","roc_auc","pr_auc","thr"]].sort_values("class")

# CSVs
per_class_csv = REPORTS/"best_model_per_class_metrics.csv"
per_class.to_csv(per_class_csv, index=False)

thr_csv = REPORTS/"best_model_thresholds.csv"
pd.DataFrame({"class": classes, "threshold": thr_vec}).to_csv(thr_csv, index=False)

print("Guardados:")
print(" -", per_class_csv)
print(" -", thr_csv)

per_class.head()


Guardados:
 - ..\reports\best_model_per_class_metrics.csv
 - ..\reports\best_model_thresholds.csv


,class,f1,roc_auc,pr_auc,thr
0,cardiovascular,0.915842,0.976815,0.970623,0.35
1,hepatorenal,0.867257,0.952166,0.937276,0.35
2,neurological,0.904841,0.971664,0.970511,0.46
3,oncological,0.854054,0.946680,0.894531,0.36


In [ ]:

from sklearn.metrics import precision_recall_curve, average_precision_score, roc_curve, auc


proba_available = 'test_proba_full' in globals()
if not proba_available and (win_dir/"model.joblib").exists():
    import joblib
    mdl = joblib.load(win_dir/"model.joblib")
    X_test = tfidf.transform(df.loc[test_idx, "text_norm"].astype(str))
    test_proba_full = to_proba(mdl, X_test)
    proba_available = True

if proba_available:
    for i, c in enumerate(classes):
        y_true = y_test[:, i]
        y_score = test_proba_full[:, i]

        # PR
        p, r, _ = precision_recall_curve(y_true, y_score)
        ap = average_precision_score(y_true, y_score)
        plt.figure(figsize=(5,4))
        plt.plot(r, p, lw=2)
        plt.xlabel("Recall"); plt.ylabel("Precision")
        plt.title(f"PR curve — {c} (AP={ap:.3f})")
        plt.tight_layout()
        plt.savefig(FIGS/f"pr_{c}.png", dpi=200)
        plt.close()

        # ROC
        fpr, tpr, _ = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(5,4))
        plt.plot(fpr, tpr, lw=2)
        plt.plot([0,1], [0,1], linestyle="--", lw=1)
        plt.xlabel("FPR"); plt.ylabel("TPR")
        plt.title(f"ROC curve — {c} (AUC={roc_auc:.3f})")
        plt.tight_layout()
        plt.savefig(FIGS/f"roc_{c}.png", dpi=200)
        plt.close()

    print("Guardadas curvas PR/ROC por clase en", FIGS)
else:
    print("No hay probabilidades de test en memoria ni modelo single para recomputar; omito curvas.")


Guardadas curvas PR/ROC por clase en ..\reports\figures


In [ ]:

if 'test_proba_full' in globals():
    y_pred = (test_proba_full >= thr_vec).astype(int)

    err_rows = []
    for i, c in enumerate(classes):
        yt, yp, ys = y_test[:, i], y_pred[:, i], test_proba_full[:, i]
        TP = int(((yt==1) & (yp==1)).sum())
        FP = int(((yt==0) & (yp==1)).sum())
        FN = int(((yt==1) & (yp==0)).sum())
        TN = int(((yt==0) & (yp==0)).sum())

        
        fp_idx = np.where((yt==0) & (yp==1))[0]
        fn_idx = np.where((yt==1) & (yp==0))[0]
        
        top_fp = fp_idx[np.argsort(-ys[fp_idx])][:3]
        top_fn = fn_idx[np.argsort(ys[fn_idx])[:3]]

        err_rows.append({
            "class": c, "TP": TP, "FP": FP, "FN": FN, "TN": TN,
            "top_fp_examples": [int(test_idx[j]) for j in top_fp.tolist()],
            "top_fn_examples": [int(test_idx[j]) for j in top_fn.tolist()],
        })

    err_df = pd.DataFrame(err_rows)
    err_csv = REPORTS/"best_model_errors_summary.csv"
    err_df.to_csv(err_csv, index=False)
    print("Guardado análisis de errores:", err_csv)
else:
    print("Sin probabilidades de test -> omito análisis de errores (Celda C puede recomputarlas si el modelo es single).")


Guardado análisis de errores: ..\reports\best_model_errors_summary.csv


In [ ]:
from textwrap import dedent

def df_to_md_safe(df):
    try:
        return df.to_markdown(index=False) 
    except Exception:
        
        header = "| " + " | ".join(df.columns.astype(str)) + " |"
        sep    = "| " + " | ".join(["---"]*len(df.columns)) + " |"
        rows   = ["| " + " | ".join(map(str, r)) + " |" for r in df.astype(str).values]
        return "\n".join([header, sep] + rows)


n_rows = len(df)
label_counts = {c: int(df[c].sum()) for c in classes}


rank_path = REPORTS/"experiments_rank.csv"
rank_md = ""
if rank_path.exists():
    rank_df = pd.read_csv(rank_path).sort_values("test_f1_macro", ascending=False)
    rank_md = df_to_md_safe(rank_df)


wm = winner_metrics
per_class = pd.DataFrame.from_dict(wm["per_class"], orient="index").reset_index().rename(columns={"index":"class"})
per_class = per_class[["class","f1","roc_auc","pr_auc","thr"]].sort_values("class")
per_class_md = df_to_md_safe(per_class)

md = f"""# Modeling Report — {winner_name}
**Fecha:** {datetime.now():%Y-%m-%d %H:%M}

## 1) Resumen ejecutivo
- **Ganador:** `{winner_name}`
- **F1 micro (test):** {wm["f1_micro"]:.4f}
- **F1 macro (test):** {wm["f1_macro"]:.4f}
- **ROC-AUC macro (test):** {wm["roc_auc_macro"]:.4f}
- **PR-AUC macro (test):** {wm["pr_auc_macro"]:.4f}

## 2) Datos
- Filas: **{n_rows}**
- Clases (**{len(classes)}**): {", ".join(classes)}
- Positivos por clase: {label_counts}

## 5) Ranking de modelos (top)
{rank_md if rank_md else "_No se encontró experiments_rank.csv_"}

## 6) Métricas por clase (ganador)
{per_class_md}

## 7) Umbrales por clase
Archivo: `reports/best_model_thresholds.csv`.
"""
out_md = REPORTS/"modeling_report.md"
with open(out_md, "w", encoding="utf-8") as f:
    f.write(md)
print("Guardado:", out_md)


Guardado: ..\reports\modeling_report.md
